In [1]:
import csv
import os
import zipfile

In [2]:
# Descompactando
with zipfile.ZipFile("data/logs completos/Messages_Traffic.zip", 'r') as zip_ref:
    zip_ref.extractall("data/logs completos/")

O tratamento consiste em extrair um formato de planilha onde cada linha tem o [id, data] de cada mensagem CAN.

In [4]:
if not os.path.exists("data/logs(id+data)"):
    os.makedirs("data/logs(id+data)")

lista_zipada = list()

for file in os.listdir("data/logs completos"):

    if file == "Messages_Traffic.zip":
        continue

    data_list = list()
    id_list = list()
    relative_time_list = list()

    with open(f"data/logs completos/{file}", mode='r', errors="ignore") as arquivo_csv:
        leitor_csv = csv.reader(arquivo_csv)
        valido = False
        for linha in leitor_csv:
            if "DW CAN 01" in linha:
                for coluna in linha[9:10]:
                    if len(coluna) == 2:
                        id_list.append(coluna)
                        valido = True
                
                if valido == True:
                    colunas = int("0", 16)
                    for coluna in linha[12:16]:
                        if coluna != "":
                            colunas += int(str(coluna), 16)
                        
                    data_list.append(colunas)
                
                    for coluna in linha[2:3]:
                        relative_time_list.append(coluna)
                    
                valido = False

    for identification in id_list:
        id_list[id_list.index(identification)] = int(identification, 16)

    lista_zipada.extend(list(zip(relative_time_list ,id_list, data_list)))

with open(f"data/logs(id+data)/Benign_Traffic(id+data).csv", mode="w", newline="") as arquivo_csv:
    escritor_csv = csv.writer(arquivo_csv)
    for elemento in lista_zipada:
        escritor_csv.writerow(elemento)